In [85]:
# The code was removed by DSX for sharing.

In [198]:
#Storing all the text in one list and sentiment score in another
for i in range(0,len(training_data_dict)):
    sentiment.append(getSentimentLabel(float(training_data_dict[i].get("sentiment score"))))
#     print(float(training_data_dict[i].get("sentiment score")),"Converted->",sentiment[i])
    q=""
    spanList=training_data_dict[i].get("spans")
    for j in range(0,len(spanList)):
        q+=training_data_dict[i].get("spans")[j]
    text.append(q)

    
    if len(q)>max_count:
        max_count=len(q)
# print(max_count)
test=[]
for i in range(0,len(test_data_dict)):
    spanList=training_data_dict[i].get("spans")
    spanList = [s.encode('utf-8') for s in spanList]
    q=""
    for j in spanList:
        q+=j
    test.append(q)



In [87]:
#Transform Y_Train
import numpy as np
# print(sentiment)
Y_Train = np.asarray(sentiment)
# print(Y_Train)


In [88]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# If encoding is not used it will throw error as parts of the string are not ascii or utf-8 compliant
text = [s.encode('utf-8') for s in text]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
_dict = tokenizer.texts_to_sequences(text)
X_Train = pad_sequences(_dict,max_count)
# X_Train = np.asarray(_dict)
# print(X_Train)




In [89]:
#Counting maximum number of features
max_feature=0
for i in X_Train:
    for j in i:
        if j>max_feature:
            max_feature=j
max_feature+=1
print(max_feature)

2624


In [101]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, MaxPooling1D,LSTM


maxlen=X_Train.shape[1]
val_split = 0.2

filters = 200
kernel_size = 2
epochs=200


In [102]:
#Final Model

print('Build model...')


model = Sequential()
model.add(Embedding(max_features, 25, input_length=maxlen))
model.add(Dropout(0.5))
model.add(Conv1D(filters,kernel_size, padding='valid', activation='sigmoid', strides=1))
model.add(MaxPooling1D(pool_size=16))
model.add(LSTM(50))
model.add(Dense(4,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation="tanh"))

model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy'])


model.summary()

model.fit(X_Train, Y_Train,      
          validation_split=val_split,
          epochs=epochs)

#Experiment 1
#filters = 300
#kernel_size = 2
#epochs=30
#cosine :0.89715011

#Experiment 2
# filters = 300
# kernel_size = 2
# epochs=100
#cosine :0.90310312

#Experiment 3
# filters = 300
# kernel_size = 2
# epochs=200
#cosine : 0.91220539

#Experiment 4
# filters = 200
# kernel_size = 2
# epochs=200
#cosine : 0.92589344

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 129, 25)           65600     
_________________________________________________________________
dropout_56 (Dropout)         (None, 129, 25)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 128, 200)          10200     
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 8, 200)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 50)                50200     
_________________________________________________________________
dense_39 (Dense)             (None, 4)                 204       
_________________________________________________________________
dropout_57 (Dropout)         (None, 4)                 0     

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

# print("Output shape: ", Y_Train.shape)
_pred = np.array(Y_Train).reshape(1,-1)
# print("Output new shape: ", _pred.shape)
predictions = model.predict(X_Train)
predictedVals = [pr[0] for pr in predictions ]
predictedVals_reshaped = np.array(predictedVals).reshape(1,-1)
# print("Predicted new shape: ",predictedVals_reshaped.shape)
similarity = cosine_similarity(_pred, predictedVals_reshaped)
print(similarity)



('Output shape: ', (1700,))
('Output new shape: ', (1, 1700))
('Predicted new shape: ', (1, 1700))
[[ 0.92589344]]


In [106]:
#Print predictions vs sentiments
# predictions=predictions.reshape(1700)
# for i in range(0,len(text)):
#     print(text[i]," -> ",float(training_data_dict[i].get("sentiment score")), " -> ",predictions[i])

('watching for bounce tomorrow', ' -> ', 0.366, ' -> ', 0.44757059)
('record number of passengers served in 2015', ' -> ', 0.638, ' -> ', 0.98580033)
('out $NFLX -.35', ' -> ', -0.494, ' -> ', -0.42721528)
('Looking for a strong bounceLunchtime rally coming', ' -> ', 0.46, ' -> ', 0.48733103)
('Very intrigued with the technology and growth potential', ' -> ', 0.403, ' -> ', 0.44200605)
('short workedputs up', ' -> ', 0.0, ' -> ', 0.4050023)
('overbought', ' -> ', -0.296, ' -> ', -0.38876703)
('absolute garbage still upstores TOTALLY EMPTYstock mispriced', ' -> ', -0.546, ' -> ', -0.91918516)
('Biggest Market Losers', ' -> ', -0.438, ' -> ', -0.43644536)
('Love this company long time.', ' -> ', 0.408, ' -> ', 0.4583782)
('$GOOG $GOOGL would suck', ' -> ', -0.398, ' -> ', -0.43435532)
("who won't pay anymoreREAL risk", ' -> ', -0.349, ' -> ', -0.43757623)
('No edge offered', ' -> ', 0.025, ' -> ', 0.42150804)
('runs into the 50sma on the acquisition news', ' -> ', 0.486, ' -> ', 0.445335

In [162]:
# training_data_dict
df = pd.DataFrame.from_dict(training_data_dict)
df['SENTIMENT SCORE (Predicted)'] = predictedVals
df=({
    "ID":df["id"],
    "SPANS":df["spans"],
    "SOURCE":df["source"],
    "CASHTAG":df["cashtag"],
    "SENTIMENT SCORE":df["sentiment score"],
"SENTIMENT SCORE (Predicted)":df['SENTIMENT SCORE (Predicted)']
})


In [206]:
test2=tokenizer.texts_to_sequences(test)
X_Test=pad_sequences(test2,max_count)
    
test_results=model.predict(X_Test)
# X_Test.shape
test_results

df_test = pd.DataFrame.from_dict(test_data_dict)
df_test['SENTIMENT SCORE (Predicted)'] = test_results

df_test

,cashtag,id,source,spans,SENTIMENT SCORE (Predicted)
0,$COST,709723193125175300,twitter,consumers keep cautious stance,0.447571
1,$CTRP,719529094652698600,twitter,Close $CTRP @46.16 from 43.55 entry +6%;$;+6%,0.985800
2,$INTC,39048670,stocktwits,Every Reason to Be Bullish,-0.427215
3,$PANW,37048093,stocktwits,$PANW need anoth $1 to all time high,0.487331
4,$JRCC,6207860,stocktwits,Long setup,0.442006
5,$SBUX,719911771625140200,twitter,will be a solid long term investmen;$;#passive...,0.405002
6,$TSLA,719571433307955200,twitter,recall,-0.388767
7,$MAR,711874651043926000,twitter,Marriott wins new Starwood deal by raising cas...,-0.919185
8,$ABBV,719559500852662300,twitter,Time To Embrace Dividend Growth,-0.436445
9,$MS,709737853639663600,twitter,Bullshit! All #AppleStores are EMPTY in #China...,0.458378


In [210]:
import pixiedust
pixiedust.enableJobMonitor()
display(df)

CASHTAG,ID,SENTIMENT SCORE,SENTIMENT SCORE (Predicted),SOURCE,SPANS
$FB,719659409228451840,0.366,0.447570592165,twitter,[u'watching for bounce tomorrow']
$LUV,719904304207962112,0.638,0.985800325871,twitter,[u'record number of passengers served in 2015']
$NFLX,5329774,-0.494,-0.427215278149,stocktwits,[u'out $NFLX -.35']
$DIA,719891468173844480,0.460,0.487331032753,twitter,"[u'Looking for a strong bounce', u'Lunchtime rally coming']"
$PLUG,20091246,0.403,0.44200605154,stocktwits,[u'Very intrigued with the technology and growth potential']
$GMCR,5819749,0.000,0.405002295971,stocktwits,"[u'short worked', u'puts up']"
$IBM,709741154393133056,-0.296,-0.388767033815,twitter,[u'overbought']
$JOSB,17892972,-0.546,-0.919185161591,stocktwits,"[u'absolute garbage still up', u'stores TOTALLY EMPTY', u'stock mispriced']"
$CSTM,709834259687710720,-0.438,-0.436445355415,twitter,[u'Biggest Market Losers']
$PYPL,708481442079068160,0.408,0.458378195763,twitter,[u'Love this company long time.']


In [208]:
display(df_test)

cashtag,id,source,spans,SENTIMENT SCORE (Predicted)
$COST,709723193125175300,twitter,consumers keep cautious stance,0.447571
$CTRP,719529094652698600,twitter,Close $CTRP @46.16 from 43.55 entry +6%;$;+6%,0.9858
$INTC,39048670,stocktwits,Every Reason to Be Bullish,-0.427215
$PANW,37048093,stocktwits,$PANW need anoth $1 to all time high,0.487331
$JRCC,6207860,stocktwits,Long setup,0.442006
$SBUX,719911771625140200,twitter,will be a solid long term investmen;$;#passive;$;Also added $SBUX today;$; #Income,0.405002
$TSLA,719571433307955200,twitter,recall,-0.388767
$MAR,711874651043926000,twitter,Marriott wins new Starwood deal by raising cash portion of bid,-0.919185
$ABBV,719559500852662300,twitter,Time To Embrace Dividend Growth,-0.436445
$MS,709737853639663600,twitter,Bullshit! All #AppleStores are EMPTY in #China;$;$MS must be preparing to liquidate $AAPL,0.458378


In [205]:
pred_norm = np.linalg.norm(predictedVals_reshaped)
print(pred_norm)
orig_norm = np.linalg.norm(_pred)
print(orig_norm)

cos_weight = pred_norm/orig_norm
print(cos_weight)

final_cos = cos_weight * similarity[0][0]
print(final_cos)

22.7473
25.5978514723
0.888641775258
0.822787591389
